# Final Project

## Predict whether a mammogram mass is benign or malignant

We'll be using the "mammographic masses" public dataset from the UCI repository (source: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

This data contains 961 instances of masses detected in mammograms, and contains the following attributes:


   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)
   
BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

A lot of unnecessary anguish and surgery arises from false positives arising from mammogram results. If we can build a better way to interpret them through supervised machine learning, it could improve a lot of lives.

## Your assignment

Apply several different supervised machine learning techniques to this data set, and see which one yields the highest accuracy as measured with K-Fold cross validation (K=10). Apply:

* Decision tree
* Random forest
* KNN
* Naive Bayes
* SVM
* Logistic Regression
* And, as a bonus challenge, a neural network using Keras.

The data needs to be cleaned; many rows contain missing data, and there may be erroneous data identifiable as outliers as well.

Remember some techniques such as SVM also require the input data to be normalized first.

Many techniques also have "hyperparameters" that need to be tuned. Once you identify a promising approach, see if you can make it even better by tuning its hyperparameters.

I was able to achieve over 80% accuracy - can you beat that?

Below I've set up an outline of a notebook for this project, with some guidance and hints. If you're up for a real challenge, try doing this project from scratch in a new, clean notebook!


## Let's begin: prepare your data

Start by importing the mammographic_masses.data.txt file into a Pandas dataframe (hint: use read_csv) and take a look at it.

In [17]:
%matplotlib inline
import pandas as pd
import numpy as np

data = pd.read_csv("./mammographic_masses.data.txt", na_values="?", names=["BI_RADS", "age", "shape", "margin", "density", "severity"])
data.head()

,BI_RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


Make sure you use the optional parmaters in read_csv to convert missing data (indicated by a ?) into NaN, and to add the appropriate column names (BI_RADS, age, shape, margin, density, and severity):

Evaluate whether the data needs cleaning; your model is only as good as the data it's given. Hint: use describe() on the dataframe.

In [18]:
data.describe()

,BI_RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


There are quite a few missing values in the data set. Before we just drop every row that's missing data, let's make sure we don't bias our data in doing so. Does there appear to be any sort of correlation to what sort of data has missing fields? If there were, we'd have to try and go back and fill that data in.

In [22]:
data.loc[(data['age'].isnull()) |
              (data['shape'].isnull()) |
              (data['margin'].isnull()) |
              (data['density'].isnull())]

,BI_RADS,age,shape,margin,density,severity
1,4.0,43.0,1.0,1.0,NaN,1
4,5.0,74.0,1.0,5.0,NaN,1
5,4.0,65.0,1.0,NaN,3.0,0
6,4.0,70.0,NaN,NaN,3.0,0
7,5.0,42.0,1.0,NaN,3.0,0
9,5.0,60.0,NaN,5.0,1.0,1
12,4.0,64.0,1.0,NaN,3.0,0
19,4.0,40.0,1.0,NaN,NaN,0
20,NaN,66.0,NaN,NaN,1.0,1
22,4.0,43.0,1.0,NaN,NaN,0


If the missing data seems randomly distributed, go ahead and drop rows with missing data. Hint: use dropna().

In [24]:
data.dropna(inplace=True)
data.describe()

,BI_RADS,age,shape,margin,density,severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.393976,55.781928,2.781928,2.813253,2.915663,0.485542
std,1.888371,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


Next you'll need to convert the Pandas dataframes into numpy arrays that can be used by scikit_learn. Create an array that extracts only the feature data we want to work with (age, shape, margin, and density) and another array that contains the classes (severity). You'll also need an array of the feature name labels.

In [28]:
x = np.array(data[["BI_RADS", "age", "shape", "margin", "density"]])
y = np.array(data["severity"])

Some of our models require the input data to be normalized, so go ahead and normalize the attribute data. Hint: use preprocessing.StandardScaler().

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)
x_norm = scaler.transform(x)
x_norm

array([[ 0.3211177 ,  0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.3211177 ,  0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-0.20875843, -1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [-0.20875843,  0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.3211177 ,  0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [-0.20875843,  0.42406719,  0.17563638,  0.11923341,  0.24046607]])

## Decision Trees

Before moving to K-Fold cross validation and random forests, start by creating a single train/test split of our data. Set aside 75% for training, and 25% for testing.

In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.25, random_state=42)

Now create a DecisionTreeClassifier and fit it to your training data.

In [48]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Display the resulting decision tree.

In [50]:
tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Measure the accuracy of the resulting decision tree model using your test data.

In [52]:
from sklearn.metrics import accuracy_score

y_pred = tree.predict(x_test)
accuracy_score(y_test, y_pred)

0.7788461538461539

Now instead of a single train/test split, use K-Fold cross validation to get a better measure of your model's accuracy (K=10). Hint: use model_selection.cross_val_score

In [67]:
from sklearn.model_selection import cross_val_score
tree_results = cross_val_score(tree, x_norm, y, cv=10)
tree_results

array([0.82142857, 0.82142857, 0.78571429, 0.80722892, 0.80722892,
       0.6746988 , 0.74698795, 0.7804878 , 0.75609756, 0.70731707])

Now try a RandomForestClassifier instead. Does it perform better?

In [68]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest_results = cross_val_score(forest, x_norm, y, cv=10)
forest_results

array([0.75      , 0.80952381, 0.80952381, 0.80722892, 0.8313253 ,
       0.73493976, 0.80722892, 0.7804878 , 0.81707317, 0.70731707])

In [69]:
print("Mean Tree: ", np.mean(tree_results))
print("Mean Forest: ", np.mean(forest_results))

Mean Tree:  0.7708618445909071
Mean Forest:  0.7854648559394372


## SVM

Next try using svm.SVC with a linear kernel. How does it compare to the decision tree?

In [82]:
from sklearn.svm import SVC
svc = SVC(gamma="auto")
svc_results = cross_val_score(svc, x_norm, y, cv=10)
svc_results

array([0.79761905, 0.85714286, 0.9047619 , 0.80722892, 0.90361446,
       0.79518072, 0.84337349, 0.76829268, 0.8902439 , 0.7195122 ])

In [83]:
print("Mean Tree: ", np.mean(tree_results))
print("Mean SVC: ", np.mean(svc_results))

Mean Tree:  0.7708618445909071
Mean SVC:  0.8286970180373061


## KNN
How about K-Nearest-Neighbors? Hint: use neighbors.KNeighborsClassifier - it's a lot easier than implementing KNN from scratch like we did earlier in the course. Start with a K of 10. K is an example of a hyperparameter - a parameter on the model itself which may need to be tuned for best results on your particular data set.

In [107]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn_results = cross_val_score(knn, x_norm, y, cv=10)

print("Mean KNN: ", np.mean(knn_results))

Mean KNN:  0.794959629458601


Choosing K is tricky, so we can't discard KNN until we've tried different values of K. Write a for loop to run KNN with K values ranging from 1 to 50 and see if K makes a substantial difference. Make a note of the best performance you could get out of KNN.

In [97]:
for k in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn_results = cross_val_score(knn, x_norm, y, cv=10)
    print("k={}: {}".format(k, np.mean(knn_results)))

k=1: 0.7443092929208122
k=2: 0.727223528259379
k=3: 0.7708328085862615
k=4: 0.7733148622364021
k=5: 0.7855526636161371
k=6: 0.7973699676755803
k=7: 0.7962522564124093
k=8: 0.7950320445545247
k=9: 0.8010424975161973
k=10: 0.794959629458601


## Naive Bayes

Now try naive_bayes.MultinomialNB. How does its accuracy stack up? Hint: you'll need to use MinMaxScaler to get the features in the range MultinomialNB requires.

In [105]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB

nb_scaler = MinMaxScaler()
nb_scaler.fit(x)

naive_bayes = MultinomialNB()
naive_bayes_results = cross_val_score(naive_bayes, nb_scaler.transform(x), y, cv=10)

print("Mean Naive Bayes: ", np.mean(naive_bayes_results))

Mean Naive Bayes:  0.7832007472398305


## Revisiting SVM

svm.SVC may perform differently with different kernels. The choice of kernel is an example of a "hyperparamter." Try the rbf, sigmoid, and poly kernels and see what the best-performing kernel is. Do we have a new winner?

In [112]:
svc_rbf = SVC(gamma="auto", kernel="rbf")
svc_rbf_results = cross_val_score(svc_rbf, x_norm, y, cv=10)
print("Mean SVC RBF: ", np.mean(svc_rbf_results))

Mean SVC RBF:  0.8286970180373061


In [113]:
svc_sig = SVC(gamma="auto", kernel="sigmoid")
svc_sig_results = cross_val_score(svc_sig, x_norm, y, cv=10)
print("Mean SVC SIG: ", np.mean(svc_sig_results))

Mean SVC SIG:  0.7650973230902706


In [116]:
svc_poly = SVC(gamma="auto", kernel="poly")
svc_poly_results = cross_val_score(svc_poly, x_norm, y, cv=10)
print("Mean SVC POLY: ", np.mean(svc_poly_results))

Mean SVC POLY:  0.8047591061108544


## Logistic Regression

We've tried all these fancy techniques, but fundamentally this is just a binary classification problem. Try Logisitic Regression, which is a simple way to tackling this sort of thing.

In [121]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver="liblinear")
logistic_results = cross_val_score(logistic, x_norm, y, cv=10)
print("Logistic: ", np.mean(logistic_results))

Logistic:  0.8251843611379315


## Neural Networks

As a bonus challenge, let's see if an artificial neural network can do even better. You can use Keras to set up a neural network with 1 binary output neuron and see how it performs. Don't be afraid to run a large number of epochs to train the model if necessary.

In [129]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

def create_model():
    model = Sequential()
    #4 feature inputs going into an 6-unit layer (more does not seem to help - in fact you can go down to 4)
    model.add(Dense(6, input_dim=5, kernel_initializer='normal', activation='relu'))
    # "Deep learning" turns out to be unnecessary - this additional hidden layer doesn't help either.
    #model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (benign or malignant)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model; adam seemed to work best
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [132]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, x_norm, y, cv=10)
cv_scores.mean()

0.8228915750980377

In [131]:
cv_scores.mean()

0.8228915750980377

## Do we have a winner?

Which model, and which choice of hyperparameters, performed the best? Feel free to share your results!